# final-ffwd

> Like BlockInternalsExperiment, but only saves the final t_i of the final ffwd layer. Useful for longer strings that would generate too much data otherwise. 


In [ ]:
#| default_exp experiments.final_ffwd

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| export
from collections import defaultdict, OrderedDict
from dataclasses import dataclass
import json
import math
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
from pathlib import Path
import tempfile
from typing import Callable, Dict, Iterable, Iterator, List, Optional, Protocol, Sequence, Tuple

In [ ]:
#| export
import click
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

In [ ]:
#| export
from transformer_experiments.common.databatcher import DataBatcher
from transformer_experiments.common.substring_generator import all_unique_substrings
from transformer_experiments.common.utils import topk_across_batches
from transformer_experiments.dataset_split import split_text_dataset
from transformer_experiments.datasets.tinyshakespeare import (
    TinyShakespeareDataSet,
)
from transformer_experiments.models.transformer import (
    block_size,
    n_embed,
    n_layer,
    TransformerLanguageModel
)
from transformer_experiments.models.transformer_helpers import (
    EncodingHelpers,
    LogitsWrapper,
    TransformerAccessors
)
from transformer_experiments.tokenizers.char_tokenizer import CharacterTokenizer
from transformer_experiments.trained_models.tinyshakespeare_transformer import (
    create_model_and_tokenizer,
)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device is {device}")

device is cpu


In [ ]:
ts = TinyShakespeareDataSet(cache_file='../artifacts/input.txt')
m, tokenizer = create_model_and_tokenizer(
    saved_model_filename='../artifacts/shakespeare.pt',
    dataset=ts,
    device=device,
)
_, val_data = split_text_dataset(ts.text, tokenizer, train_pct=0.9, device=device)
encoding_helpers = EncodingHelpers(tokenizer, device)
accessors = TransformerAccessors(m, device)

In [ ]:
# | export
class FinalFFWDExperiment:
    def __init__(
        self,
        eh: EncodingHelpers,
        accessors: TransformerAccessors,
        strings: Sequence[str],
        output_dir: Path,
        batch_size: int = 10000,
    ):
        self.eh = eh
        self.accessors = accessors
        self.strings = strings
        self.output_dir = output_dir
        self.batch_size = batch_size

        # Create a map of string to index to enable fast lookup.
        self.idx_map = OrderedDict((s, idx) for idx, s in enumerate(self.strings))

        self.n_batches = math.ceil(len(self.strings) / self.batch_size)

    def sample_length(self) -> int:
        return len(self.strings[0])

    def run(self, disable_progress_bars: bool = False):
        for batch_idx in tqdm(range(self.n_batches), disable=disable_progress_bars):
            start_idx = batch_idx * self.batch_size
            end_idx = start_idx + self.batch_size
            batch_strings = self.strings[start_idx:end_idx]
            self._run_batch(batch_idx, batch_strings)

    def _ffwd_output_filename(self, batch_idx: int, block_idx: int) -> Path:
        return self.output_dir / f'ffwd_output-{batch_idx:04d}-{block_idx:02d}.pt'

    def _run_batch(self, batch_idx: int, batch_strings: Sequence[str]):
        tokens = self.eh.tokenize_strings(batch_strings)
        embeddings = self.accessors.embed_tokens(tokens)

        # Run the embeddings through the model.
        _, io_accessors = self.accessors.run_model(embeddings)

        # Write the result of the final block's final t_i to disk.
        block_idx = n_layer - 1
        torch.save(
            io_accessors[block_idx].output('ffwd')[:, -1, :].clone(),
            self._ffwd_output_filename(batch_idx, block_idx),
        )

In [ ]:
# Test for FinalFFWDExperiment
s_len = 3
strings = all_unique_substrings(ts.text[:100], s_len)

with tempfile.TemporaryDirectory() as tmpdirname:
    output_dir = Path(tmpdirname)
    experiment = FinalFFWDExperiment(
        encoding_helpers, accessors, strings, output_dir=output_dir, batch_size=10
    )
    test_eq(experiment.sample_length(), s_len)
    experiment.run(disable_progress_bars=True)

    # Test the expected files exist
    block_idx = n_layer - 1
    for batch_idx in range(experiment.n_batches):
        test_eq(experiment._ffwd_output_filename(batch_idx, block_idx).exists(), True)

In [ ]:
#| export
@click.command()
@click.argument("model_weights_filename", type=click.Path(exists=True))
@click.argument("dataset_cache_filename", type=click.Path(exists=True))
@click.argument("output_folder", type=click.Path(exists=True))
@click.option(
    "-s",
    "--sample_len",
    required=True,
    type=click.IntRange(min=1, max=block_size),
)
@click.option(
    "-m",
    "--max_batch_size",
    required=False,
    type=click.IntRange(min=1),
    default=10000,
)
def run(
    model_weights_filename: str,
    dataset_cache_filename: str,
    output_folder: str,
    sample_len: int,
    max_batch_size: int,
):
    click.echo(f"Running block internals experiment for with:")
    click.echo(f"  model weights: {model_weights_filename}")
    click.echo(f"  dataset cache: {dataset_cache_filename}")
    click.echo(f"  output folder: {output_folder}")
    click.echo(f"  sample length: {sample_len}")
    click.echo(f"  max batch size: {max_batch_size}")

    # Instantiate the model, tokenizer, and dataset
    device = "cuda" if torch.cuda.is_available() else "cpu"
    click.echo(f"device is {device}")

    ts = TinyShakespeareDataSet(cache_file=dataset_cache_filename)
    m, tokenizer = create_model_and_tokenizer(
        saved_model_filename=model_weights_filename,
        dataset=ts,
        device=device,
    )

    strings = all_unique_substrings(ts.text, sample_len)

    encoding_helpers = EncodingHelpers(tokenizer, device)
    accessors = TransformerAccessors(m, device)

    # Create the experiment
    exp = FinalFFWDExperiment(
        encoding_helpers, accessors, strings, Path(output_folder), max_batch_size
    )

    exp.run()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()